# Import libraries

In [ ]:
import psycopg2
import pandas as pd
import time

# Connect to DB

In [ ]:
# Connect to PostgreSQL database
db_params = {
    "host": "rain.db.elephantsql.com",
    "dbname": "auspovuc",  # Replace with your desired database name
    "user": "auspovuc",  # Replace with your PostgreSQL username
    "password": "bmJdG19Daw9rkEsJ3VnkefRGCBF_oy7F",  # Replace with your PostgreSQL password
}

conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# Create Views - Ranked genre Per age_group Per condition 

In [ ]:
start = time.process_time()

# Declare conditions
conditions = ['anxiety', 'depression', 'insomnia', 'ocd']

# Declare age_group
groups = ['early_years', 'young_adults', 'mature_adults', 'middle_age', 'elderly']

# Create Views for each condition and age_group, ranking genres
for i in range(len(conditions)):
    for n in range(len(groups)):
        query = f"""
            CREATE OR REPLACE VIEW {groups[n]}_{conditions[i]}_genre AS(
                SELECT a.genre, COUNT(*),
                  ROW_NUMBER() OVER(ORDER BY COUNT(*) DESC) AS row_n
                FROM (
                  SELECT s.respondent, g.*
                  FROM survey s
                  CROSS JOIN LATERAL (
                     values (s.freq1, 'freq1'),
                            (s.freq2, 'freq2'),
                            (s.freq3, 'freq3')
                        ) as g(genre, frequency)
                WHERE s.{conditions[i]} >= 7
                AND age_group = '{groups[n]}') AS a
                WHERE a.genre IS NOT NULL
                GROUP BY a.genre
                )
                """
        cur.execute(query,)
        conn.commit()

print(time.process_time() - start)

# Check Views Created

In [ ]:
# Check that views were created
query = f"""
            SELECT table_name
            FROM information_schema.views
            WHERE table_schema NOT IN ('pg_catalog', 'information_schema')
            AND table_name !='pg_stat_statements'
            ORDER BY table_name
            """
cur.execute(query,)
conn.commit()
results = cur.fetchall()
print(*results, sep='\n')

# Close connection

In [ ]:
# close connection to db
conn.close()